In [ ]:
from math import pi
import numpy as np
from pde import plot_magnitudes,PDE,movie_scalar,UnitGrid,plot_magnitudes, CartesianGrid, MemoryStorage, ScalarField, plot_kymograph, plot_kymographs, FieldCollection, PlotTracker, UnitGrid, PDEBase, ExplicitSolver, FileStorage,VectorField
import pandas as pd
import os,sys
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.colors import LogNorm
from pde.tools.numba import jit
import matplotlib.backends.backend_pdf
import numba as nb
import pickle

In [ ]:
datafolder = 'data/'
datafile = 'test'
filename = datafolder + datafile
with open(filename, 'rb') as output:
    dic = pickle.load(output)
    phi = dic['phi']
    dphi = dic['dphi']
    alpha = dic['alpha']
    L = dic['L']
    N= dic['N']
    frames=len(phi)
rephi = np.real(phi)
imphi = np.imag(phi)

redphi = np.real(dphi)
imdphi = np.imag(dphi)

argphi = np.angle(phi)
modphi = np.abs(phi)

argdphi = np.angle(dphi)
moddphi = np.abs(dphi)

Po =  np.real(dphi)*np.imag(phi) - np.imag(dphi) *np.real(phi) 
Pd = np.abs(dphi)**2
sumPo = np.sum(Po,axis=(1))
sumPd = np.sum(Pd,axis=(1))


normPo = Po/sumPo[:,None]
normPd = Pd/sumPd[:,None]
dP = Po-Pd
sumdP = sumPo - sumPd
normdP = dP/sumdP[:,None]
density = modphi**2


times = np.arange(frames)
momentum = np.abs(np.sum(phi,axis=(1)))**2
energy = np.sum(density,axis=(1))
error = momentum/energy


In [ ]:
plt.rcParams.update({'font.size': 16})
fig,ax = plt.subplots(4,5,figsize=(21,16),dpi=150)

imqties = [rephi,modphi,argphi,density,energy,
           Po,normPo,sumPo,np.sum(modphi,axis=(1)),np.log10(energy),
           Pd,normPd,sumPd,error,
           dP,normdP,sumdP]

titles =['Re($\\phi$)','$|\\phi|$','Arg($\\phi$)','$|\\phi|^2$','$\\int|\\phi|$',
           'odd power: Im($\\phi$)Re($\\dot{\\phi}$)-Im($\\dot{\\phi}$)Re($\\phi$)','$\\frac{P_{\\mathrm{odd}}}{\\int P_{\\mathrm{odd}}}$','$\\int P_{\mathrm{odd}}$','$\\int(|\\phi|)$','$\\mathrm{log}(|\\phi|^2)$',
           'dissipated power: $|\\dot{\\phi}|^2$','$\\frac{P_{\\mathrm{disp}}}{\\int P_{\\mathrm{disp}}}$','$\\int P_{\mathrm{disp}}$','$|\\int \\phi|^2/\\int |\\phi|^2$',
            '$P_{\\mathrm{odd}}-P_{\\mathrm{disp}}$','$\\frac{P_{\\mathrm{odd}}-P_{\\mathrm{disp}}}{\\int P_{\\mathrm{odd}}-P_{\\mathrm{disp}}}$','$\\int P_{\\mathrm{odd}}-P_{\\mathrm{disp}}$']
            
fig.suptitle('1D phi model')
sc = [0,1,2,3,5,6,10,11,15,16]
sct = [4,7,8,9,12,13,17]

o=0
ims=[]
X = np.arange(N)
for n,i in enumerate(ax.flatten()):
    if n in sc:
        ims.append(i.scatter(X,imqties[o][0],s=5))
        i.set_title(titles[o])
        o+=1
    elif n in sct:
        ims.append(i.scatter(times[0],imqties[o][0],s=5))
        i.set_title(titles[o])
        o+=1
plt.tight_layout()

def animate(i):
    print(i)
    o=0
    for n,j in enumerate(ax.flatten()):
        if n in sc:
            ims[o].set_offsets(np.c_[X,imqties[o][i]])
            ymin = np.min(imqties[o][i])
            ymax = np.max(imqties[o][i])
            j.set_ylim([ymin,ymax])
            o+=1
        elif n in sct:
            ims[o].set_offsets(np.c_[times[:i],imqties[o][:i]])
            if i>1:
                j.set_xlim([np.min(times[:i]),np.max(times[:i])])
                j.set_ylim([np.min(imqties[o][:i]),np.max(imqties[o][:i])])
            o+=1
        
anim = animation.FuncAnimation(fig, animate, 
                               frames=frames, interval=20, blit=False)
anim.save('output/test.mp4')